In [1]:
import pandas as pd

In [2]:
filename = r"C:\Users\Иннокентий\Downloads\train.tsv"

In [3]:
sep='>'

In [4]:
df = pd.read_csv(filename, sep='\t', encoding='utf-8', names=['text', 'with_sep'])

In [5]:
df

,text,with_sep
0,Ri numam xkamik are taq kʼo jumuchʼ lajuj ujun...,Ri nu>mam x>kamik are taq kʼo jumuchʼ lajuj u>...
1,Nojel qʼij ri a Tiʼx kuyuqʼuj le taq awaj jela...,Nojel qʼij ri a Tiʼx k>u>yuqʼuj le taq awaj je...
2,Kateʼbʼin .,K>at>eʼ>bʼin .
3,Entonses are kʼwaʼ le kawaj kinbʼij pan in .,Entonses are kʼ>waʼ le k>aw>aj k>in>bʼij pan in .
4,Nim raqan le cheʼ .,Nim r>aqan le cheʼ .
...,...,...
1467,Xax rij mansaʼn .,Xax r>ij mansaʼn .
1468,Man kʼo ta pwaq chech ri umuqubʼal .,Man kʼo ta pwaq ch>ech ri u>muqubʼal .
1469,Rajawaxik kakʼaman chawe le utijik le lej .,Rajawaxik ka>kʼaman ch>aw>e le u>tij>ik le lej .
1470,Sipojinaq ri nupam .,Sipojinaq ri nu>pam .


In [6]:
kiche_anbur = "a [a], ä [ə], e [ɛ], i [i], o [o], u [u], m [m], n [n], nh [ŋ], p [p], bʼ [ɓ], t [t], tʼ [tʼ], k [k], kʼ [kʼ], q [q], qʼ [qʼ], tz [ʦ], tzʼ [ʦ’], ch [ʧ], chʼ [ʧ’], s [s], x [ʃ], j [χ], h [h], r [r], l [l], y [j], w [w], ʼ [ʔ]"
alphabet = []
for letter in kiche_anbur.split(","):
    letter = letter.split()[0]
    alphabet.append(letter)
    alphabet.append(letter[0].upper() + letter[1:])

In [7]:
print(alphabet)

['a', 'A', 'ä', 'Ä', 'e', 'E', 'i', 'I', 'o', 'O', 'u', 'U', 'm', 'M', 'n', 'N', 'nh', 'Nh', 'p', 'P', 'bʼ', 'Bʼ', 't', 'T', 'tʼ', 'Tʼ', 'k', 'K', 'kʼ', 'Kʼ', 'q', 'Q', 'qʼ', 'Qʼ', 'tz', 'Tz', 'tzʼ', 'Tzʼ', 'ch', 'Ch', 'chʼ', 'Chʼ', 's', 'S', 'x', 'X', 'j', 'J', 'h', 'H', 'r', 'R', 'l', 'L', 'y', 'Y', 'w', 'W', 'ʼ', 'ʼ']


In [8]:
alph_to_int = dict(zip(alphabet, range(len(alphabet))))
int_to_alph = dict(zip(range(len(alphabet)), alphabet))

In [9]:
def window(text, size):
    tokens = []
    i = 0
    while i < (len(text)-size+1):
        try:
            current = ''.join(text[i:i+size])
        except:
            current = 'not_in_alphabet'
        if current in alphabet:
            tokens.append(alph_to_int[current])
            i += size
        else:
            tokens.append(text[i])
            i += 1
    return tokens

In [10]:
def tokenize(text):
    tokens = list(text + 'ггг')
    for i in range(3, 0, -1):
        tokens = window(tokens, i)
    tokens_2 = []
    for token in tokens:
        if token in int_to_alph:
            token = int_to_alph[token]
        tokens_2.append(token)
    return tokens_2

In [11]:
tokenize("Entzʼonses are kʼwa")

['E',
 'n',
 'tzʼ',
 'o',
 'n',
 's',
 'e',
 's',
 ' ',
 'a',
 'r',
 'e',
 ' ',
 'kʼ',
 'w',
 'a']

In [12]:
#b, m и s значат beginning, middle и end, а s — single
def to_labels(text):
    border_tokens = sep + ".!,? "   #тут можно расширить каким-нибудь string.punctuation например
    labels = text.copy()
    labels[0] = 'b-morph'
    for i in range(len(text)-1):
        current = labels[i]
        previous = labels[i-1]
        next_ = labels[i+1]
        if text[i] in border_tokens:
            next_ = 'b-morph'
            if previous != 'b-morph':
                previous = 'e-morph'
            else:
                previous = 's-morph'
        else:
            if current != 'b-morph':
                current = 'm-morph'
        labels[i] = current
        labels[i-1] = previous
        labels[i+1] = next_
    if labels[-1] != 'b-morph':
        labels[-1] = 'e-morph'
    else:
        labels[-1] = 's-morph'
    return labels

In [13]:
df['tokens'] = df['text'].apply(tokenize)

In [14]:
df['labels'] = df['tokens'].apply(to_labels)

In [15]:
df

,text,with_sep,tokens,labels
0,Ri numam xkamik are taq kʼo jumuchʼ lajuj ujun...,Ri nu>mam x>kamik are taq kʼo jumuchʼ lajuj u>...,"[R, i, , n, u, m, a, m, , x, k, a, m, i, k, ...","[b-morph, e-morph, , b-morph, m-morph, m-morp..."
1,Nojel qʼij ri a Tiʼx kuyuqʼuj le taq awaj jela...,Nojel qʼij ri a Tiʼx k>u>yuqʼuj le taq awaj je...,"[N, o, j, e, l, , qʼ, i, j, , r, i, , a, ,...","[b-morph, m-morph, m-morph, m-morph, e-morph, ..."
2,Kateʼbʼin .,K>at>eʼ>bʼin .,"[K, a, t, e, ʼ, bʼ, i, n, , .]","[b-morph, m-morph, m-morph, m-morph, m-morph, ..."
3,Entonses are kʼwaʼ le kawaj kinbʼij pan in .,Entonses are kʼ>waʼ le k>aw>aj k>in>bʼij pan in .,"[E, n, t, o, n, s, e, s, , a, r, e, , kʼ, w,...","[b-morph, m-morph, m-morph, m-morph, m-morph, ..."
4,Nim raqan le cheʼ .,Nim r>aqan le cheʼ .,"[N, i, m, , r, a, q, a, n, , l, e, , ch, e,...","[b-morph, m-morph, e-morph, , b-morph, m-morp..."
...,...,...,...,...
1467,Xax rij mansaʼn .,Xax r>ij mansaʼn .,"[X, a, x, , r, i, j, , m, a, n, s, a, ʼ, n, ...","[b-morph, m-morph, e-morph, , b-morph, m-morp..."
1468,Man kʼo ta pwaq chech ri umuqubʼal .,Man kʼo ta pwaq ch>ech ri u>muqubʼal .,"[M, a, n, , kʼ, o, , t, a, , p, w, a, q, ,...","[b-morph, m-morph, e-morph, , b-morph, e-morp..."
1469,Rajawaxik kakʼaman chawe le utijik le lej .,Rajawaxik ka>kʼaman ch>aw>e le u>tij>ik le lej .,"[R, a, j, a, w, a, x, i, k, , k, a, kʼ, a, m,...","[b-morph, m-morph, m-morph, m-morph, m-morph, ..."
1470,Sipojinaq ri nupam .,Sipojinaq ri nu>pam .,"[S, i, p, o, j, i, n, a, q, , r, i, , n, u, ...","[b-morph, m-morph, m-morph, m-morph, m-morph, ..."


In [16]:
import pickle

In [17]:
filename = filename.split('\\')[-1]
filename = filename.split('.')[0]

In [18]:
with open (filename + '.pickle', 'bw') as f:
    pickle.dump(df, f)